<a href="https://colab.research.google.com/github/adamdavidcole/stylegan3-fun-blend/blob/main/blend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN3 Network Blending GUI

A user interface for experimenting with StyleGAN3 network blending. If you're looking for StyleGAN2 blending, check out [this notebook](https://github.com/adamdavidcole/stylegan2-ada-pytorch-adam/blob/main/network_blending_gui.ipynb).

Select your source and destination models and play with various blend settings and sliders.

## Setup Libraries and Google Drive Connection

In [ ]:
# Check GPU connection
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-8174d7d1-6c95-7509-eef8-76f5df686e11)


In [ ]:
# Connect Google Drive
# WARNING: only run this if you'd like to save the results in your gdrive!

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
!pip install gdown --upgrade

if os.path.isdir("/content/drive/MyDrive/stylegan3-fun-blend"):
    %cd "/content/drive/MyDrive/stylegan3-fun-blend"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !git clone https://github.com/adamdavidcole/stylegan3-fun-blend.git
    %cd stylegan3-fun-blend
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    # !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/adamdavidcole/stylegan3-fun-blend.git
    %cd stylegan3-fun-blend
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained

In [ ]:
# ONLY IF NECESSARY: pull new code files into drive repo 
# !git config --global user.name "test"
# !git config --global user.email "test@test.com"
# !git fetch origin
# !git pull
# !git stash
# !git checkout origin/main -- "*.py" 
# !git checkout origin/main -- "*.ipynb"

Updating 59cff72..ffc78c4
Fast-forward
^C
Saved working directory and index state WIP on main: ffc78c4 end of day save


In [ ]:
!pip install einops ninja gdown

In [ ]:
import numpy as np
from datetime import datetime
import torch
import dnnlib
from dnnlib.util import format_time
import legacy
import PIL.Image

from torch_utils import gen_utils



## Helper Functions & Setup

In [ ]:
#common functions 
import pickle, torch, PIL, copy, cv2, math
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
from io import BytesIO
from PIL import Image, ImageEnhance

from IPython.display import Image as DisplayImage, clear_output

# define device to use
device = torch.device('cuda')

def get_model(path):
  # with open(path, 'rb') as f:
  #   _G = pickle.load(f)['G_ema'].cuda()
  device = torch.device('cuda')
  with dnnlib.util.open_url(path) as fp:
      _G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)
  
  return _G

#tensor to PIL image 
def t2i(t):
  return PIL.Image.fromarray((t*127.5+127).clamp(0,255)[0].permute(1,2,0).cpu().numpy().astype('uint8'))

#stack an array of PIL images horizontally
def add_imgs(images):
  widths, heights = zip(*(i.size for i in images))

  total_width = sum(widths)
  max_height = max(heights)

  new_im = PIL.Image.new('RGB', (total_width, max_height))

  x_offset = 0
  for im in images:
    new_im.paste(im, (x_offset,0))
    x_offset += im.size[0]
  return new_im


def apply_mask(matrix, mask, fill_value):
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    return masked.filled()
 
def apply_threshold(matrix, low_value, high_value):
    low_mask = matrix < low_value
    matrix = apply_mask(matrix, low_mask, low_value)
 
    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)
 
    return matrix

# A simple color correction script to brighten overly dark images
def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100
 
    half_percent = percent / 200.0
 
    channels = cv2.split(img)
 
    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2
        # find the low and high precentile values (based on the input percentile)
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)
 
        assert len(flat.shape) == 1
 
        flat = np.sort(flat)
 
        n_cols = flat.shape[0]
 
        low_val  = flat[math.floor(n_cols * half_percent)-1]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))-1]
 
 
        # saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # scale the channel
        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)
 
    return cv2.merge(out_channels)
 
def normalize(inf, thresh):
    img = np.array(inf)
    out_img = simplest_cb(img, thresh)
    return PIL.Image.fromarray(out_img)

def get_w_from_path(w_path):
  projected_w_np = np.load(projected_w_path)[0]
  w = torch.tensor(projected_w_np).to(device).unsqueeze(0)
  return w

def synthesize_tensor_from_w(G, w):
  # print(w.shape)
  # print(w)
  return G.synthesis(w, noise_mode='const', force_fp32=True)

def synthesize_img_from_w(G, w):
  tensor = synthesize_tensor_from_w(G, w)
  return t2i(tensor)

def synthesize_tensor_from_w_path(G, w_path):
  w = get_w_from_path(w_path)
  return synthesize_tensor_from_w(G, w)

def synthesize_img_from_w_path(G, w_path):
  tensor = synthesize_tensor_from_w_path(G, w_path)
  return t2i(tensor)

def synthesize_img_from_w_path(G, w_path):
  tensor = synthesize_tensor_from_w_path(G, w_path)
  return t2i(tensor)

def synthesize_img_from_w_np(G, w_np):
  w = torch.tensor(w_np).to(device).unsqueeze(0)
  tensor = synthesize_img_from_w(G, w)
  return t2i(tensor)

def synthesize_img_from_seed(G, seed):
  rnd = np.random.RandomState(seed)
  z = torch.tensor(rnd.randn(1,G.z_dim)).cuda()

  w = G.mapping(z, None, truncation_psi=0.7, truncation_cutoff=8)
  return synthesize_img_from_w(G, w)

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Model Selection

**Select a source and destination model.** 

Keep in mind that the destination model needs to be fine-tuned from the source model for the blend to work. 

We will use `FFHQ_R_1024` -> `METFACES_R_1024` for this example, but feel free to paste in links to other pairs of models you'd like to expriment with.

In [ ]:
#@title Select Source and Destination Networks  {run: "auto"}
#Download pretrained checkpoint

# StyleGAN 256 "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl"
# StyleGAN 1024 "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl" 

source_model = "FFHQ_R_1024" #@param ["FFHQU_R_1024", "FFHQ_R_1024", "FFHQU_T_1024", "FFHQ_T_1024", "FFHQU_R_256", "FFHQ_R_256", "FFHQU_T_256", "FFHQ_T_256"] {allow-input: true}
# destination_model = "/content/stylegan3-r-metfaces-1024x1024.pkl" #@param ["BUTTERFLY_256", "BUTTERFLY_256_HALF_TRAINED", "KISS_HD", "pokemon_120", "BUTTERFLY_0012", "BUTTERFLY_0013"] {allow-input: true}
destination_model = "METFACES_R_1024" #@param ["METFACES_R_1024"] {allow-input: true}

model_keys = {
    "FFHQU_R_1024": "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl",
    "FFHQU_T_1024": "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhqu-1024x1024.pkl",
    "FFHQ_R_1024": "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl",
    "FFHQ_T_1024": "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl",

    "FFHQU_R_256": "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl",
    "FFHQU_T_256": "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhqu-256x256.pkl",
    "FFHQ_R_256": "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-256x256.pkl",
    "FFHQ_T_256": "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-256x256.pkl",


    # "BUTTERFLY_256": "/content/drive/MyDrive/stylegan3/results/00033-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6/network-snapshot-000283.pkl",
    # "BUTTERFLY_256_HALF_TRAINED": "/content/drive/MyDrive/stylegan3/results/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6/network-snapshot-000020.pkl",
    # "BUTTERFLY_0012": "/content/drive/MyDrive/stylegan3/results/00009-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6/network-snapshot-000012.pkl",
    # "BUTTERFLY_0013": "/content/drive/MyDrive/stylegan3/results/00010-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6/network-snapshot-000001.pkl",


    'METFACES_R_1024': 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-metfaces-1024x1024.pkl',

    # "KISS_HD": "/content/drive/MyDrive/stylegan3/results/00048-stylegan3-r-kiss_hd_square-gpus1-batch8-gamma6.6/network-snapshot-001216.pkl",

    # "pokemon_120": "/content/drive/MyDrive/stylegan3-fun-blend/results/pokemon/training/00003-stylegan3-r-pokemon_256-gpus1-batch32-gamma6.6-resume_custom/network-snapshot-000120.pkl"
}

# source_model_pkl = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl" #@param  {type:"string"}
# destination_model_pkl = "/content/drive/MyDrive/stylegan3/results/00033-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6/network-snapshot-000283.pkl" #@param {type: "string"}

source_model_pkl = model_keys[source_model] if source_model in model_keys else source_model
destination_model_pkl = model_keys[destination_model] if destination_model in model_keys else destination_model

G = get_model(source_model_pkl)
G_new = copy.deepcopy(G)
G_tuned = get_model(destination_model_pkl)

## Optional: Image File Selection and Projection

If you'd like to project an image of a real face, follow the steps below. This process is experimental and results are often unsatisfactory.

In [ ]:
#@title Select or Upload an Image to Project {run: "auto"}
#@markdown (Note press the play button first to see the upload option)
projection_source_images_outdir = "projection_source_images"
projection_source_vectors_outdir = "projection_source_vectors"

if not os.path.isdir(projection_source_images_outdir):
  !mkdir -p $projection_source_images_outdir

def upload_files():
  filepaths = []
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    k = k.replace(" ", "_")
    filepath = f"{projection_source_images_outdir}/{k}"
    open(filepath, 'wb').write(v)
    filepaths.append(filepath)
  return list(filepaths)[0]


button = widgets.Button(description="Upload New File")
output = widgets.Output()

img_file_path = "" #@param {type: "string"}

def on_button_clicked(b):
  global img_file_path
  # Display the message within the output widget.
  # with output:
  img_file_path = upload_files();
  clear_output(wait=True)
  display(DisplayImage(img_file_path))

  print(color.BOLD + color.YELLOW + "NOTE: " + color.END + color.END +  "to save this upload for future runs, copy the line below into img_file_path param in the form");
  print("\t" + img_file_path)

  print("");
  display(button, output)

if img_file_path:
  display(DisplayImage(img_file_path))

button.on_click(on_button_clicked)
display(button, output)

Button(description='Upload New File', style=ButtonStyle())

Output()

### OPTIONAL: Image Enhancements
Use these sliders to improve the image quality or amplify specific colors before projection.

In [ ]:
#@title Image enhancement options {run: "auto"}
#@markdown (Click **"Save Image"** when done)
img = Image.open(img_file_path)
img_out = img

color_factor = 1.2 #@param {type: "slider", min:0, max:5, step: 0.1}
contrast_factor = 1.2 #@param {type: "slider", min:0, max:5, step: 0.1}
sharpness_factor = 1.2 #@param {type: "slider", min:0, max:5, step: 0.1}
brightness_factor = 1 #@param {type: "slider", min:0, max:5, step: 0.1}

red_factor = 1 #@param {type: "slider", min:0, max:5, step: 0.01}
green_factor = 1 #@param {type: "slider", min:0, max:5, step: 0.01}
blue_factor = 1 #@param {type: "slider", min:0, max:5, step: 0.01}

img_out = ImageEnhance.Color(img_out).enhance(color_factor)
img_out = ImageEnhance.Contrast(img_out).enhance(contrast_factor)
img_out = ImageEnhance.Sharpness(img_out).enhance(sharpness_factor)
img_out = ImageEnhance.Brightness(img_out).enhance(brightness_factor)

img_out_np = np.array(img_out).astype(np.float64)
img_out_np[:,:,0] *= red_factor
img_out_np[:,:,1] *= green_factor
img_out_np[:,:,2] *= blue_factor

img_out_np = np.clip(img_out_np, 0, 255)
img_out = Image.fromarray(img_out_np.astype(np.uint8))


display(add_imgs([img,img_out]))

def on_button_clicked(b):
  global img_file_path
  head, tail = os.path.split(img_file_path)
  name, ext = os.path.splitext(tail)
  new_file_path = f"{head}/{name}_enhanced{ext}"

  img_out.save(new_file_path)

  img_file_path = new_file_path

  print(color.BOLD + color.YELLOW + "NOTE: " + color.END + color.END +  "to use this image in future runs, copy the line below into img_file_path param in the form above");
  print("\t" + new_file_path)

button = widgets.Button(description="Save Image")
button.on_click(on_button_clicked)
print("")
display(button, output)



### Projection Into GAN Space

Generate a projection into the GAN latent by either:
  1. Entering a path to an existing projection numpy file
  2. Running the projection script on the image selected above

**Note:** run *either* step 1 or step 2 (not both)

In [ ]:
#@title EITHER: Select a path to an existing projection {run: "auto"}
projected_w_path = "" #@param {type: "string"}

# if projected_w_path:
#   im1 = synthesize_img_from_w_path(G, projected_w_path)
#   im2 = synthesize_img_from_w_path(G_tuned, projected_w_path)
#   im = add_imgs([im1, im2])
#   display(im)

if projected_w_path:
  projected_w_np = np.load(projected_w_path)
  print(projected_w_np.shape)
  projected_w =  torch.tensor(projected_w_np).to(device)
  synth_tensor = G_new.synthesis(projected_w, noise_mode='const')
  img = PIL.Image.fromarray((synth_tensor*127.5+127).clamp(0,255)[0].permute(1,2,0).cpu().numpy().astype('uint8'))
# t2i(synth)
# synth_image = gen_utils.w_to_img(G, dlatents=projected_w, noise_mode='const')[0]
# img = PIL.Image.fromarray(synth, 'RGB')

  display(img)

In [ ]:
#@title OR: Create a new projection from selected photo
#@markdown (This process can take over 10 minuets )
projection_source_vectors_outdir = "projection_source_vectors"

num_steps =  1000#@param {type: "integer", min:0, max:10000, step:1 }
trunc = 0.2 #@param {type: "slider", min:0, max:1, step:0.01 }
loss_paper = "sgan2" #@param ['sgan2', 'im2sgan', 'discriminator', 'clip']

# --save-video off --project-in-wplus
!python projector.py --outdir=$projection_source_vectors_outdir --target=$img_file_path --trunc=$trunc --loss-paper=$loss_paper --num-steps=$num_steps --project-in-wplus --stabilize-projection \
    --cfg=stylegan3-r --network=$source_model_pkl
print("Projection complete! \n\n")

last_projected_source_vector_dir= os.listdir(projection_source_vectors_outdir)[-1]
last_projected_source_vector_path = f"{projection_source_vectors_outdir}/{last_projected_source_vector_dir}"

projected_w_filename = [f for f in os.listdir(last_projected_source_vector_path) if f.endswith('.npy')][0]
projected_w_path = f"{last_projected_source_vector_path}/{projected_w_filename}"

print(f"Projected w at {projected_w_path}")

np

im1 = synthesize_img_from_w_path(G, projected_w_path)
im2 = synthesize_img_from_w_path(G_tuned, projected_w_path)
im = add_imgs([im1, im2])

im.save(f"{last_projected_source_vector_path}/proj_w_tuned.jpg")

display(im)

### EXPERIMENTAL: Laten Vector Tuning
Experiment with shifting along random latent directions. This ended up not having much use for the blending application but is interesting to play with.

In [ ]:
#@title fine tune projected W {run: "auto"}
fine_tune_random_seed = 660 #@param {type: "slider", min:0, max: 1000, step: 1}
fine_tune_magnitude = -0.00632  #@param {type: "slider", min:-5, max: 5, step: 0.00001 }
w = get_w_from_path(projected_w_path)

torch.manual_seed(fine_tune_random_seed)
print(torch.rand(w.shape, device=device))
w_fine_tuned = w + ((torch.rand(w.shape, device=device)-0.5)*2) * fine_tune_magnitude

# print(w_fine_tuned)

img0 = synthesize_img_from_w(G, w)
img1 = synthesize_img_from_w(G, w_fine_tuned)
img2 = synthesize_img_from_w(G_tuned, w_fine_tuned)
display(add_imgs([img0, img1, img2]))
# np_arr = w.cpu().detach().numpy()


## Interactive Blending
Below are three different UIs for blending between the source and destination networks increasing in complexity and control. 

I recommend starting with the EZ Blend options to get a sense of the results and then moving down to Fine Tuned Blending for more controlled outputs. The final Overblending UI is experimental.

Select `use_projected_w` in the forms below *only* if you've run through the projections steps above.

### Setup Blend Function

In [ ]:
#Blend using mask. Number of layers in stylegan3 depends on config and not on gen resolution, as it was with stylegan2.

# blend = [0,0,0,0,0,0,0,0.2,0.5,0.7,0.9,1,1,1,1] # 15
# blend = [0,0,0,0,0,0,0,0.2,0.5,0.7,0.8,.8,.8,.8,.8] # 15
# blend = [0,0,0,0,0,0.2,0.2,0.2,0.5,0.7,0.8,.8,.8,.8,1]
# blend = [0]*7+[0.8]*(15-7)

# Not blending affine layers gives us colors closer to the original gen, without affecting the geometry much.
def doBlend(blend_mask, blend_affine_layers): 
  newDictSynt = G_tuned.synthesis.state_dict().copy()
  # newDictSynt = G.synthesis.state_dict().copy()
  GSyntKeys = G.synthesis.state_dict().keys()

  for key in GSyntKeys:
    # print(key)

    if key[:1]!='L': continue
    
    l = blend_mask[int(key.split('_')[0][1:])]
    
    if not blend_affine_layers:
      if 'affine' in key: l = 0
    
    # print(key)
    newDictSynt[key] = G_tuned.synthesis.state_dict()[key]*l + G.synthesis.state_dict()[key]*(1-l)
    # newDictSynt[key] = G.synthesis.state_dict()[key]


  G_new.synthesis.load_state_dict(newDictSynt)
  # print(G_new)

# doBlend()

### EZ Blending 

Simple blend functions between the source and desination models. Select a seed value and blend mode.

In [ ]:
#@title  { run: "auto" }
#@markdown **Select source vector**
use_projected_w = False #@param {type:"boolean"}
#@markdown If not using a projected w, select a seed
seed =  2782 #@param {type:"slider", min:0, max:10000, step:1}
only_display_blended =  False #@param {type:"boolean"}

#@markdown --- 
#@markdown **Play with network belnding sliders** <br/>
#@markdown (first sliders control lower level features, last sliders contol higher level features)


# blend_thresh = 0 #@param {type:"slider", min:0, max:13, step:1}

# Standard Blend (0-1)
blend_affine_layers = True #@param {type:"boolean"}
psi = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}

# blend_val_0 = 0.2 #@param {type:"slider", min:0, max:1, step:0.001}
# blend_val_1 = 0.19 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_2 = 0.2 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_3 = 0.2 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_4 = 0.47 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_5 = 0.47 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_6 = 0.47 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_7 = 0.47 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_8 = 0.48 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_9 = 0.48 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_10 = 0.48 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_11 = 0.8 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_12 = 0 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_13 = 0 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_14 = 0 #@param {type:"slider", min:0, max:1, step:0.01}

blend_mode = "coarse_medium_fine" #@param ["20%", "50%", "70%", "halfway_cross_over", "halfway_cross_over_inverse", "coarse_medium_fine", "coarse_medium_fine_inverse", "ramp_up", "ramp_down"]

blend_modes = {
    "20%": [0.2, 0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2, 0.2, 0.2],
    "50%": [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,],
    "70%": [0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7],

    "halfway_cross_over": [0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],
    "halfway_cross_over_inverse": [1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],


    "coarse_medium_fine": [0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,1,1,1,1,1],
    "coarse_medium_fine_inverse": [1,1,1,1,1,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0],


    "ramp_up": [0,0,0,0,0,0.2,0.2,0.2,0.5,0.7,0.8,.8,.8,.8,1],
    "ramp_down": [1,1,1,1,1,0.8,0.8,0.8,0.5,0.3,0.2,.2,.2,.2,0]
}

# blend = [0,0,0,0,0,0.2,0.2,0.2,0.5,0.7,0.8,.8,.8,.8,1]
blend = blend_modes[blend_mode]

if use_projected_w:
  w = get_w_from_path(projected_w_path)
  print(f"W: {projected_w_path}")

else:
  bl_str = ('_').join([str(o) for o in blend])
  net = destination_model_pkl.split('/')[-1]
  rnd = np.random.RandomState(seed)
  z = torch.tensor(rnd.randn(1,G.z_dim)).cuda()

  w = G.mapping(z, None, truncation_psi=psi, truncation_cutoff=8)

  print(f"Seed: {seed}, Psi: {psi}")


doBlend(blend, blend_affine_layers)

print(blend)

if only_display_blended:
  im = G_new.synthesis(w, noise_mode='const', force_fp32=True)
  im = t2i(im)
  display(im)
else:
  im1 = G.synthesis(w, noise_mode='const', force_fp32=True)
  im2 = G_tuned.synthesis(w, noise_mode='const', force_fp32=True)
  im3 = G_new.synthesis(w, noise_mode='const', force_fp32=True)

  im1 = t2i(im1)
  im2 = t2i(im2)
  im3 = t2i(im3)
  im = add_imgs([im1, im3, im2])
  display(im)
# im.save(f'/content/m{net}_psi{psi}_b{bl_str}_s{seed}.jpg')

### Fine Tune Blending

Fine tuned blending allows you to control the individual blend levels between the source and destination models. Higher sliders will control coarser, structual behavior while lower sliders control finer details like color and texture.

In [ ]:
#@title { run: "auto" }
#@markdown **Select source vector**
use_projected_w = False #@param {type:"boolean"}
#@markdown If not using a projected w, select a seed
seed =  9539 #@param {type:"slider", min:0, max:10000, step:1}
only_display_blended =  False #@param {type:"boolean"}

#@markdown --- 
#@markdown **Play with network belnding sliders** <br/>
#@markdown (first sliders control lower level features, last sliders contol higher level features)


# blend_thresh = 0 #@param {type:"slider", min:0, max:13, step:1}

# Standard Blend (0-1)
blend_affine_layers = True #@param {type:"boolean"}
blend_val_0 = 0 #@param {type:"slider", min:0, max:1, step:0.001}
blend_val_1 = 0 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_2 = 0 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_3 = 0 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_4 = 0 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_5 = 0.46 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_6 = 0.46 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_7 = 0.46 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_8 = 0.46 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_9 = 0.46 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_10 = 1 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_11 = 1 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_12 = 1 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_13 = 1 #@param {type:"slider", min:0, max:1, step:0.01}
blend_val_14 = 1 #@param {type:"slider", min:0, max:1, step:0.01}



# blend = [0,0,0,0,0,0.2,0.2,0.2,0.5,0.7,0.8,.8,.8,.8,1]
blend = [blend_val_0, blend_val_1, blend_val_2, blend_val_3, blend_val_4, blend_val_5, blend_val_6, blend_val_7, blend_val_8, blend_val_9, blend_val_10, blend_val_11, blend_val_12, blend_val_13, blend_val_14]
psi = 0.75 #@param {type:"slider", min:0, max:1, step:0.01}

if use_projected_w:
  w = get_w_from_path(projected_w_path)
else:
  bl_str = ('_').join([str(o) for o in blend])
  net = destination_model_pkl.split('/')[-1]
  rnd = np.random.RandomState(seed)
  z = torch.tensor(rnd.randn(1,G.z_dim)).cuda()

  w = G.mapping(z, None, truncation_psi=psi, truncation_cutoff=8)

print(blend)
doBlend(blend, blend_affine_layers)

if only_display_blended:
  im = G_new.synthesis(w, noise_mode='const', force_fp32=True)
  im = t2i(im)
  display(im)
else:
  im1 = G.synthesis(w, noise_mode='const', force_fp32=True)
  im2 = G_tuned.synthesis(w, noise_mode='const', force_fp32=True)
  im3 = G_new.synthesis(w, noise_mode='const', force_fp32=True)

  im1 = t2i(im1)
  im2 = t2i(im2)
  im3 = t2i(im3)
  im = add_imgs([im1, im3, im2])
  display(im)
# im.save(f'/content/m{net}_psi{psi}_b{bl_str}_s{seed}.jpg')

### Experimental Over Blend
Overblending is a technique where you go beyond the sensible values of 0 and 1 when blending between the source and destination. The results often collapse when the sliders go far beyond the [0-1] scale, but it's fun to play with!

In [ ]:
#@title { run: "auto" }
#@markdown **Select source vector**
use_projected_w = False #@param {type:"boolean"}
#@markdown If not using a projected w, select a seed
seed =  753 #@param {type:"slider", min:0, max:10000, step:1}

#@markdown --- 
#@markdown **Play with network belnding sliders** <br/>
#@markdown (first sliders control lower level features, last sliders contol higher level features)


# blend_thresh = 0 #@param {type:"slider", min:0, max:13, step:1}

# Standard Blend (-10-10)
# blend_val_0 = 1 #@param {type:"slider", min:0, max:1, step:0.001}
# blend_val_1 = 0.45 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_2 = 0.45 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_3 = 0.45 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_4 = 0.48 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_5 = 0.46 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_6 = 1 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_7 = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_8 = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_9 = 0.76 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_10 = 0.89 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_11 = 1 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_12 = 1 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_13 = 1 #@param {type:"slider", min:0, max:1, step:0.01}
# blend_val_14 = 1 #@param {type:"slider", min:0, max:1, step:0.01}

# Experimental Blend (0-1)
blend_affine_layers = True #@param {type:"boolean"}
blend_val_0 = -0.321 #@param {type:"slider", min:-10, max:10, step:0.001}
blend_val_1 = -1.08 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_2 = -1.1 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_3 = -0.74 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_4 = -0.26 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_5 = -0.51 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_6 = -0.76 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_7 = -0.76 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_8 = 0 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_9 = 0.94 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_10 = 1.26 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_11 = 1.74 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_12 = 1.59 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_13 = 1.16 #@param {type:"slider", min:-10, max:10, step:0.01}
blend_val_14 = 0.62 #@param {type:"slider", min:-10, max:10, step:0.01}


# blend = [0,0,0,0,0,0.2,0.2,0.2,0.5,0.7,0.8,.8,.8,.8,1]
blend = [blend_val_0, blend_val_1, blend_val_2, blend_val_3, blend_val_4, blend_val_5, blend_val_6, blend_val_7, blend_val_8, blend_val_9, blend_val_10, blend_val_11, blend_val_12, blend_val_13, blend_val_14]
blend_standard = np.clip(blend, 0.0,1.0)

psi = 0.5 #@param {type:"slider", min:-10, max:10, step:0.01}

if use_projected_w:
  w = get_w_from_path(projected_w_path)

  print(f"W: {'/'.join(projected_w_path.split('/')[-2:])}")
else:
  bl_str = ('_').join([str(o) for o in blend])
  net = destination_model_pkl.split('/')[-1]
  rnd = np.random.RandomState(seed)
  z = torch.tensor(rnd.randn(1,G.z_dim)).cuda()

  w = G.mapping(z, None, truncation_psi=psi, truncation_cutoff=8)
  print(f"Seed: {seed}")

print(blend)
print(blend_standard)

doBlend(blend, blend_affine_layers)

im1 = G.synthesis(w, noise_mode='const', force_fp32=True)
im2 = G_tuned.synthesis(w, noise_mode='const', force_fp32=True)
im3 = G_new.synthesis(w, noise_mode='const', force_fp32=True)

doBlend(blend_standard, blend_affine_layers)
im4 = G_new.synthesis(w, noise_mode='const', force_fp32=True)

im1 = t2i(im1)
im2 = t2i(im2)
im3 = t2i(im3)
im4 = t2i(im4)
im = add_imgs([im1, im3, im4, im2])
# im.save(f'/content/m{net}_psi{psi}_b{bl_str}_s{seed}.jpg')
im

## Experimental: Pix2Pix Prep
Use the blended network you've fine tuned above to create image pairs for pix2pix training.

This concept is a work in progress and has yet to be proven as effective.

In [ ]:
from tqdm import tqdm

key = "face2butterfly_local"
gen_type = "test"
pix2pix_dir = f"pix2pix_prep/{key}/{gen_type}"

# net 16 0-10000
# net 12 10000 - 12000


seed_start = 20000
img_count =  40000

if not os.path.isdir(pix2pix_dir):
  !mkdir -p $pix2pix_dir


for i in tqdm(range(img_count)):

  seed = seed_start + i

  # rnd = np.random.RandomState(seed)
  # z = torch.tensor(rnd.randn(1,G.z_dim)).cuda()

  # w = G.mapping(z, None, truncation_psi=psi, truncation_cutoff=8)
  # img = G_new.synthesis(w, noise_mode='const', force_fp32=True)
  imgA = synthesize_img_from_seed(G, seed)
  imgB = synthesize_img_from_seed(G_tuned, seed)
  img_pair = add_imgs([imgB,imgA])

  img_pair.save(f"{pix2pix_dir}/{seed}.jpg")

100%|██████████| 40000/40000 [56:01<00:00, 11.90it/s]


## Refrences

This notebook was created by [Adam Cole](https://www.instagram.com/adamcole.studio/) with a specific focus on building a user interface around network blending for artists to experiment with.

### Sources
- This code lives in a fork of [StyleGAN3-fun](https://github.com/PDillis/stylegan3-fun) by [@PDillis](https://github.com/Sxela) and we take advantage of their projection script and utilities
- Setup code was based on [@dvschultz](https://github.com/dvschultz) [StyleGAN notebooks](https://github.com/dvschultz/stylegan2-ada-pytorch) 
- The idea to use a "blend mask" and many helper functions were taken fully from [@Sxela](https://github.com/PDillis) [stylegan3_blending](https://github.com/Sxela/stylegan3_blending) repo
- Much of this work was inspired by Justin Pinkney's blogpost on [network blending](https://www.justinpinkney.com/stylegan-network-blending/)